In [ ]:
!pip install neo4j
!pip3 install tomlkit
!pip3 install -U neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=570d814ff6d6b8e94efae6125f9f3acdd1eb34bb026eb4555704d0b82bb6a126
  Stored in directory: /root/.cache/pip/wheels/30/db/9b/2cfde1fa33145219c0322f299b604daf5aba2ed443a7ed5f07
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase

database_name = "neo4j"
username = "neo4j"
password = "12345678"
uri = "bolt://localhost:7689/apan5400" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")


Successfully connected to Neo4j!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/combined_data_pulkit.csv')

print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-17-dc7b96a329b8>:5: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/combined_data_pulkit.csv')


   rating                                              title parent_asin  \
0     5.0  Tyger Auto T1 Soft Roll Up Truck Bed Tonneau C...  B0BV88374L   
1     5.0  Tailgate Insert Letters Compatible with F150 R...  B0B2WGS5ND   
2     1.0  uxcell Adhesive 3" Round Side Rearview Blind S...  B00A0GV20Q   
3     5.0  Crevelle Custom Fits 2009-2020 Nissan 370Z / 3...  B08C27WWVG   
4     5.0  Pre-Painted Trunk Spoiler Compatible with 2009...  B0719J5ZNY   

                        user_id  average_rating  rating_number  price  \
0  AHMUTK5MAIDB7NQZQYH5NIOPT6BQ             4.8         5239.0  207.0   
1  AFE337D2J37YRU5U6MVTVKNDKWDA             5.0            1.0    NaN   
2  AGMVX6M2O2J7L63ATNZA2KBWHIAQ             3.7            8.0    NaN   
3  AHQOKY3AQJG5WK4QGPCIVRA2UEBQ             4.4           65.0  95.95   
4  AEZBUG7X75XAZ6MSPQ43QWY2WBBQ             4.6           28.0    NaN   

              store                                         categories  \
0        Tyger Auto  ['Automot

In [ ]:
query =

In [ ]:

cypher_query = '''
UNWIND $products AS product
MERGE (p:Product {asin: product.parent_asin})
  ON CREATE SET p.title = product.title, p.price = product.price
MERGE (u:User {id: product.user_id})
MERGE (u)-[:PURCHASED {rating: product.rating}]->(p)
MERGE (s:Store {name: product.store})
MERGE (p)-[:SOLD_BY]->(s)
WITH product, p
UNWIND product.categories AS category
MERGE (c:Category {name: category})
MERGE (p)-[:BELONGS_TO]->(c)
'''

products = df.to_dict('records')
execute_query(driver, cypher_query, parameters={'products': products})

print("Graph creation in Neo4j is complete.")

driver.close()

In [ ]:
def execute_query(driver, query, parameters=None):
    """
    Executes a Cypher query using the provided Neo4j driver.

    :param driver: Neo4j driver object for database connection
    :param query: A string representing the Cypher query to execute
    :param parameters: A dictionary of parameters for the Cypher query
    """

In [ ]:
#
def execute_query(cypher_query):
    with driver.session() as session:
        results = session.run(cypher_query)
        return [record for record in results]

most_popular_category_query = """
MATCH (p:Product)-[:BELONGS_TO]->(c:Category)
RETURN c.name AS Category, COUNT(p) AS ProductCount
ORDER BY ProductCount DESC
LIMIT 1
"""

# Execute the queries
most_popular_category = execute_query(most_popular_category_query)

print("Most popular category:", most_popular_category)


ServiceUnavailable: Couldn't connect to localhost:7689 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7689)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7689, 0, 0)) (reason [Errno 99] Cannot assign requested address)

In [ ]:
#The plan:

#Product Nodes: Each product can be a node, with properties like title, average_rating, price, etc.
#User Nodes: Each user can also be a node, with a user_id property.
#Category Nodes: Categories can be nodes as well, with the category name as a property.
#Store Nodes: Stores are also nodes, with the store name as a property.
#PURCHASED Relationships: Connect User nodes to Product nodes to represent a purchase. The rating could be a property of this relationship.
#BELONGS_TO Relationship: Connect Product nodes to Category nodes to represent the categories each product belongs to.
#SOLD_BY Relationship: Connect Product nodes to Store nodes to represent the store selling the product.